In [2]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Conv1D, MaxPooling1D, Dropout,
    BatchNormalization, Activation, GlobalAveragePooling1D
)

In [3]:
# Custom convolution block

def conv_layers(x, filters, kernel_size, strides=1, is_train=True):
    x = Conv1D(filters=filters, kernel_size=kernel_size, strides=strides, padding='same')(x)
    if is_train:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

In [4]:
# Load ECG data

data = pd.read_csv(r"D:\JN\data set\created dataset\PCG\Resample_pcg_data_2048_channel_3.csv",header=None)  # shape: (1975, 2048)
data.shape

(1975, 2048)

In [5]:
X = data.values.reshape((-1, 2048, 1))  # shape: (1975, 2048, 1)
X.shape

(1975, 2048, 1)

In [6]:
# Build the CNN model (5 layers)

inputs = Input(shape=(2048, 1))

# 1
layer = conv_layers(inputs, filters=16, kernel_size=5, strides=1, is_train=True)
layer = MaxPooling1D(pool_size=5)(layer)
layer = Dropout(0.2, seed=1)(layer)

# 2
layer = conv_layers(layer, filters=16, kernel_size=5, strides=1, is_train=True)
layer = MaxPooling1D(pool_size=5)(layer)
layer = Dropout(0.2, seed=1)(layer)

# 3
layer = conv_layers(layer, filters=16, kernel_size=5, strides=1, is_train=True)
layer = MaxPooling1D(pool_size=5)(layer)
layer = Dropout(0.2, seed=1)(layer)

# 4
layer = conv_layers(layer, filters=32, kernel_size=5, strides=1, is_train=True)
layer = MaxPooling1D(pool_size=4)(layer)
layer = Dropout(0.2, seed=1)(layer)

# 5
layer = conv_layers(layer, filters=32, kernel_size=5, strides=1, is_train=True)
layer = MaxPooling1D(pool_size=4)(layer)
layer = Dropout(0.2, seed=1)(layer)

# Global Pooling to get 128-dimensional feature vector
layer = GlobalAveragePooling1D()(layer)  # Output shape: (1975, 128)

In [7]:
# Create and run the model

feature_model = Model(inputs=inputs, outputs=layer)
feature_model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 2048, 1)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d (Conv1D)                      │ (None, 2048, 16)            │              96 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 2048, 16)            │              64 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation (Activation)              │ (None, 2048, 16)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d (MaxPooling1D)         │ (None, 409, 16)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 409, 16)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_1 (Conv1D)                    │ (None, 409, 16)             │           1,296 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 409, 16)             │              64 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_1 (Activation)            │ (None, 409, 16)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_1 (MaxPooling1D)       │ (None, 81, 16)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 81, 16)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_2 (Conv1D)                    │ (None, 81, 16)              │           1,296 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 81, 16)              │              64 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_2 (Activation)            │ (None, 81, 16)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_2 (MaxPooling1D)       │ (None, 16, 16)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 16, 16)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_3 (Conv1D)                    │ (None, 16, 32)              │           2,592 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 16, 32)              │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_3 (Activation)            │ (None, 16, 32)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 10,880 (42.50 KB)

 Trainable params: 10,656 (41.62 KB)

 Non-trainable params: 224 (896.00 B)

In [8]:
# Extract features
features = feature_model.predict(X)  # shape: (1975, 128)

62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step


In [9]:
# Save to CSV
np.savetxt(r"D:\JN\data set\created dataset\PCG\new PCG extracted feature\new_PCG(32)_features_channel_3.csv", features, delimiter=",")
print("Feature extraction complete. Saved to new_PCG_features_channel_3.csv")

Feature extraction complete. Saved to new_PCG_features_channel_3.csv
